<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_night_20250825/blob/main/tibame20250921.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import json
import urllib.request as req
import bs4 as bs

def download_meta(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1758273800.A.03F.html"
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)

    metas = html.find_all("span", {"class":"article-meta-value"})
    uid = metas[0]
    board = metas[1]
    title = metas[2]
    posttime = metas[3]


    uid_text = uid.get_text().strip()
    board_text = board.get_text().strip()
    title_text = title.get_text().strip()
    posttime_text = posttime.get_text().strip()

    pushes = html.find_all("div", {"class":"push"})
    push_list = []
    for p in pushes:
        trans = {"推":1, "→":0, "噓":-1}
        pushtag = p.find("span", {"class":"push-tag"})
        pushuid = p.find("span", {"class":"push-userid"})
        pushcontent = p.find("span", {"class":"push-content"})
        pushipdatetime = p.find("span", {"class":"push-ipdatetime"})

        pushtag_text = pushtag.get_text().strip()
        pushtag_score = trans[pushtag_text]
        pushuid_text = pushuid.get_text().strip()
        pushcontent_text = pushcontent.get_text().strip().replace(": ", "")
        pushipdatetime_text = pushipdatetime.get_text().strip()

        push_meta = {
            "score": pushtag_score,
            "uid": pushuid_text,
            "content": pushcontent_text,
            "ipdatetime": pushipdatetime_text
        }
        push_list.append(push_meta)

    # extract: 我都放最後做(因為刪了就刪了)
    maincontent = html.find("div", {"id":"main-content"})
    targets = html.find_all("div", {"class":"article-metaline"})
    for t in targets:
        t.extract()
    targets = html.find_all("div", {"class":"article-metaline-right"})
    for t in targets:
        t.extract()
    targets = html.find_all("div", {"class":"push"})
    for t in targets:
        t.extract()
    maincontent_text = maincontent.get_text()

    # userid/看板名稱/標題/時間  推虛文: 推需>/userid/內容/ipdatetime
    row = {
        "uid": uid_text,
        "board": board_text,
        "title": title_text,
        "time": posttime_text,
        "content": maincontent_text,
        "pushes": push_list
    }

    dn = url.split("/")[-1]
    fn = dn + "/" + "metadata.json"
    print("Save to:", fn)
    # 如果資料夾不存在, 就make起來
    if not os.path.exists(dn):
        os.makedirs(dn)

    with open(fn, "w", encoding="utf-8") as f:
        json.dump(row, f, ensure_ascii=False, indent=4)



```
當你沒有class/id可以篩選的時候
1. 抓包他的區塊
2. 自己寫
```



In [ ]:
url = "https://www.ptt.cc/bbs/Beauty/M.1758273800.A.03F.html"
resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)

allow_subs = ["jpg", "jpeg", "png", "gif", "webp"]
links = html.find_all("a")
for l in links:
    l_href = l["href"]
    l_subname = l_href.split(".")[-1]
    if l_subname.lower() in allow_subs:
        print("download:", l_href)
    else:
        print("pass:", l_href)

In [10]:
imgurl = "https://i.imgur.com/xoqr8vi.jpeg"
resp = req.urlopen(imgurl)
content = resp.read()
print(content)

HTTPError: HTTP Error 429: Unknown Error